In [27]:
import numpy as np
import pandas as pd

test = pd.DataFrame({
    'period': ['2001-2005', '2006-2010'] * 5,
    'node': ['A', 'B', 'C', 'D', 'E'] * 2,
    'rank': [1, 2, 3, 4, None, 2, 1, 5, 4, 3]
})

# Method 2: Learn from observed rank changes
def impute_from_transitions(test, n_samples=1000):
    """
    Observe: A,B,C,D all stayed the same rank
    This suggests rank stability
    E is rank 5 in period 2, likely rank 5 in period 1
    """
    # Get matched observations
    matched = test.dropna()
    # Only keep nodes present in both periods
    nodes_p1 = set(matched[matched['period'] == '2001-2005']['node'])
    nodes_p2 = set(matched[matched['period'] == '2006-2010']['node'])
    common_nodes = nodes_p1 & nodes_p2

    p1 = matched[(matched['period'] == '2001-2005') & (matched['node'].isin(common_nodes))].sort_values('node')
    p2 = matched[(matched['period'] == '2006-2010') & (matched['node'].isin(common_nodes))].sort_values('node')
    
    # Calculate rank changes
    rank_changes = p2['rank'].values - p1['rank'].values
    
    print("Observed rank changes:", rank_changes)
    print("Mean change:", rank_changes.mean())
    print("Std change:", rank_changes.std())
    
    # E is rank 5 in period 2
    # If mean change is ~0, E was likely rank 5 in period 1
    # Sample from Normal(5 - mean_change, std_change)
    
    e_rank_period2 = test[(test['node'] == 'E') & (test['period'] == '2006-2010')]['rank'].values[0]
    predicted_rank_p1 = e_rank_period2 - rank_changes.mean()
    
    # Generate samples with uncertainty
    samples = np.random.normal(
        loc=predicted_rank_p1,
        scale=max(rank_changes.std(), 0.5),  # min std for uncertainty
        size=n_samples
    )
    
    # Constrain to valid ranks [1, 5]
    samples = np.clip(np.round(samples), 1, 5)
    
    return samples, rank_changes

samples, changes = impute_from_transitions(test)
print(f"\nE's imputed rank in 2001-2005:")
print(f"Posterior mean: {samples.mean():.2f}")
print(f"95% CI: [{np.percentile(samples, 2.5):.0f}, "
      f"{np.percentile(samples, 97.5):.0f}]")

Observed rank changes: [1. 1. 2. 0.]
Mean change: 1.0
Std change: 0.7071067811865476

E's imputed rank in 2001-2005:
Posterior mean: 2.06
95% CI: [1, 3]


In [30]:

# Method 3: Full Bayesian Model
# Model: rank_t ~ rank_{t-1} + change
# where change ~ Normal(mu_change, sigma_change)

def bayesian_imputation_mcmc(test, n_samples=5000):
    """
    Bayesian model:
    - Prior on rank changes: Normal(0, σ) 
    - Observe 4 rank changes (all 0)
    - Update beliefs about rank stability
    - Impute E's period 1 rank given E's period 2 rank = 5
    """
    
    # Observed data
    matched = test.dropna()
    # Only keep nodes present in both periods
    nodes_p1 = set(matched[matched['period'] == '2001-2005']['node'])
    nodes_p2 = set(matched[matched['period'] == '2006-2010']['node'])
    common_nodes = nodes_p1 & nodes_p2

    p1 = matched[(matched['period'] == '2001-2005') & (matched['node'].isin(common_nodes))].sort_values('node')
    p2 = matched[(matched['period'] == '2006-2010') & (matched['node'].isin(common_nodes))].sort_values('node')
    observed_changes = p2['rank'].values - p1['rank'].values
    
    # Priors
    mu_change_prior = 0  # Expect no change on average
    sigma_change_prior = 2  # But allow for movement
    
    # Posterior for change distribution (conjugate normal)
    n_obs = len(observed_changes)
    
    # Update posterior for mean change
    posterior_precision = 1/sigma_change_prior**2 + n_obs/1
    posterior_mean = (mu_change_prior/sigma_change_prior**2 + 
                     observed_changes.sum()/1) / posterior_precision
    posterior_std = np.sqrt(1/posterior_precision)
    
    print(f"Posterior for rank change distribution:")
    print(f"  Mean: {posterior_mean:.3f}")
    print(f"  Std: {posterior_std:.3f}")
    
    # Now impute E's period 1 rank
    # E_rank_p2 = E_rank_p1 + change
    # E_rank_p1 = E_rank_p2 - change
    
    e_rank_p2 = test[(test['node'] == 'E') & (test['period'] == '2006-2010')]['rank'].values[0]
    
    # Sample changes from posterior
    change_samples = np.random.normal(
        posterior_mean, 
        posterior_std, 
        n_samples
    )
    
    # Impute period 1 ranks
    e_rank_p1_samples = e_rank_p2 - change_samples
    
    # Constrain to valid range
    e_rank_p1_samples = np.clip(e_rank_p1_samples, 1, 5)
    
    return e_rank_p1_samples, posterior_mean, posterior_std

samples, post_mu, post_sigma = bayesian_imputation_mcmc(test)

print(f"\nE's imputed rank in 2001-2005:")
print(f"Posterior mean: {samples.mean():.2f}")
print(f"Posterior median: {np.median(samples):.0f}")
print(f"95% Credible Interval: [{np.percentile(samples, 2.5):.1f}, "
      f"{np.percentile(samples, 97.5):.1f}]")

# Probability E was rank 2
prob_rank_2 = (samples == 2).mean()
print(f"P(E was rank 2 in period 1) = {prob_rank_2:.2%}")

Posterior for rank change distribution:
  Mean: 0.941
  Std: 0.485

E's imputed rank in 2001-2005:
Posterior mean: 2.06
Posterior median: 2
95% Credible Interval: [1.1, 3.0]
P(E was rank 2 in period 1) = 0.00%


# With Scores

In [31]:
import numpy as np
import pandas as pd
from scipy import stats

# Your data structure
data = pd.DataFrame({
    'period': ['2001-2005', '2006-2010'] * 5,
    'node': ['A', 'B', 'C', 'D', 'E'] * 2,
    'score': [120, 0, 450, 30, None, 150, 0, 520, 45, 200]  # Example
})

def fit_zero_inflated_model(scores):
    """
    Fit zero-inflated log-normal to observed scores
    Returns: (prob_zero, mu_log, sigma_log) for non-zero part
    """
    scores = np.array(scores)
    scores = scores[~np.isnan(scores)]
    
    # Estimate probability of zero
    p_zero = (scores == 0).mean()
    
    # Fit log-normal to non-zero scores
    nonzero = scores[scores > 0]
    if len(nonzero) > 0:
        mu_log = np.log(nonzero).mean()
        sigma_log = np.log(nonzero).std()
    else:
        mu_log, sigma_log = 0, 1
    
    return p_zero, mu_log, sigma_log

def bayesian_score_imputation(data, n_samples=5000):
    """
    Full Bayesian imputation using score information
    """
    # Separate periods
    p1 = data[data['period'] == '2001-2005'].set_index('node')
    p2 = data[data['period'] == '2006-2010'].set_index('node')
    
    # Get matched observations
    matched_nodes = p1.index.intersection(p2.index)
    matched_p1 = p1.loc[matched_nodes, 'score'].dropna()
    matched_p2 = p2.loc[matched_nodes, 'score'].dropna()
    
    # Fit distributions for each period
    print("Fitting period 1 distribution...")
    p1_params = fit_zero_inflated_model(p1['score'])
    print(f"  P(zero) = {p1_params[0]:.2f}")
    print(f"  Non-zero: log-normal(μ={p1_params[1]:.2f}, σ={p1_params[2]:.2f})")
    
    print("\nFitting period 2 distribution...")
    p2_params = fit_zero_inflated_model(p2['score'])
    print(f"  P(zero) = {p2_params[0]:.2f}")
    print(f"  Non-zero: log-normal(μ={p2_params[1]:.2f}, σ={p2_params[2]:.2f})")
    
    # Model score transition for matched entities
    # Simple approach: score_p1 = alpha * score_p2 + noise
    # (accounts for scale difference)
    
    valid_matches = matched_p1.index.intersection(matched_p2.index)
    if len(valid_matches) > 1:
        s1 = matched_p1[valid_matches].values
        s2 = matched_p2[valid_matches].values
        
        # Filter to non-zero pairs for regression
        nonzero_mask = (s1 > 0) & (s2 > 0)
        if nonzero_mask.sum() > 1:
            # Linear regression in log space
            log_s1 = np.log(s1[nonzero_mask] + 1)
            log_s2 = np.log(s2[nonzero_mask] + 1)
            
            slope, intercept, r_val, _, _ = stats.linregress(log_s2, log_s1)
            residual_std = np.std(log_s1 - (slope * log_s2 + intercept))
            
            print(f"\nScore transition model (log scale):")
            print(f"  log(score_p1) = {slope:.3f} * log(score_p2) + {intercept:.3f}")
            print(f"  Residual std: {residual_std:.3f}")
            print(f"  R²: {r_val**2:.3f}")
        else:
            # Fallback: assume proportional scaling
            slope = 500 / 600  # Scale ratio
            intercept = 0
            residual_std = 1.0
    
    # Impute missing scores
    results = {}
    
    for node in data['node'].unique():
        if pd.isna(p1.loc[node, 'score']) and not pd.isna(p2.loc[node, 'score']):
            # Node missing in period 1, present in period 2
            score_p2 = p2.loc[node, 'score']
            
            if score_p2 == 0:
                # If zero in P2, likely zero in P1 too
                # Sample from Bernoulli then from non-zero dist
                prob_was_zero = p1_params[0] / (p1_params[0] + (1-p1_params[0]) * 0.3)
                samples = []
                for _ in range(n_samples):
                    if np.random.rand() < prob_was_zero:
                        samples.append(0)
                    else:
                        samples.append(np.exp(np.random.normal(
                            p1_params[1], p1_params[2]
                        )))
                samples = np.array(samples)
            else:
                # Non-zero in P2: use transition model
                log_s2 = np.log(score_p2 + 1)
                predicted_log_s1 = slope * log_s2 + intercept
                
                # Sample with uncertainty
                log_s1_samples = np.random.normal(
                    predicted_log_s1,
                    residual_std * 1.5,  # Inflate for missing data
                    n_samples
                )
                samples = np.exp(log_s1_samples) - 1
                samples = np.clip(samples, 0, 500)  # Respect period 1 range
            
            results[node] = {
                'period_2_score': score_p2,
                'imputed_samples': samples,
                'mean': samples.mean(),
                'median': np.median(samples),
                'ci_low': np.percentile(samples, 2.5),
                'ci_high': np.percentile(samples, 97.5)
            }
    
    return results

# Run imputation
results = bayesian_score_imputation(data)

# Display results
for node, result in results.items():
    print(f"\nNode {node}:")
    print(f"  Observed P2 score: {result['period_2_score']:.1f}")
    print(f"  Imputed P1 score: {result['mean']:.1f} "
          f"[{result['ci_low']:.1f}, {result['ci_high']:.1f}]")

# Convert to ranks (for each sample)
def samples_to_rank_distribution(data, results, n_samples=5000):
    """Convert score samples to rank distributions"""
    p1 = data[data['period'] == '2001-2005'].set_index('node')
    
    rank_samples = {node: [] for node in results.keys()}
    
    for sample_idx in range(n_samples):
        # Get scores for this sample
        scores = {}
        for node in data['node'].unique():
            if not pd.isna(p1.loc[node, 'score']):
                scores[node] = p1.loc[node, 'score']
            elif node in results:
                scores[node] = results[node]['imputed_samples'][sample_idx]
        
        # Rank them (higher score = better rank)
        sorted_nodes = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        ranks = {node: rank+1 for rank, (node, score) in enumerate(sorted_nodes)}
        
        for node in results.keys():
            rank_samples[node].append(ranks[node])
    
    return rank_samples

rank_distributions = samples_to_rank_distribution(data, results)

for node, ranks in rank_distributions.items():
    print(f"\nNode {node} - Rank distribution in period 1:")
    print(f"  Mean rank: {np.mean(ranks):.1f}")
    print(f"  Median rank: {np.median(ranks):.0f}")
    print(f"  95% CI: [{np.percentile(ranks, 2.5):.0f}, "
          f"{np.percentile(ranks, 97.5):.0f}]")

Fitting period 1 distribution...
  P(zero) = 0.25
  Non-zero: log-normal(μ=4.90, σ=0.94)

Fitting period 2 distribution...
  P(zero) = 0.20
  Non-zero: log-normal(μ=4.99, σ=1.03)

Score transition model (log scale):
  log(score_p1) = 0.800 * log(score_p2) + 0.990
  Residual std: 0.148
  R²: 0.975

Node E:
  Observed P2 score: 200.0
  Imputed P1 score: 191.2 [119.4, 293.5]

Node E - Rank distribution in period 1:
  Mean rank: 2.0
  Median rank: 2
  95% CI: [2, 3]


In [32]:
import numpy as np
import pandas as pd
from scipy import stats

# Your data structure
data = pd.DataFrame({
    'period': ['2001-2005', '2006-2010'] * 5,
    'node': ['A', 'B', 'C', 'D', 'E'] * 2,
    'score': [120, 0, 450, 30, None, 150, 0, 520, 45, 200]  # Example
})

def fit_zero_inflated_model(scores):
    """
    Fit zero-inflated log-normal to observed scores
    Returns: (prob_zero, mu_log, sigma_log) for non-zero part
    """
    scores = np.array(scores)
    scores = scores[~np.isnan(scores)]
    
    # Estimate probability of zero
    p_zero = (scores == 0).mean()
    
    # Fit log-normal to non-zero scores
    nonzero = scores[scores > 0]
    if len(nonzero) > 0:
        mu_log = np.log(nonzero).mean()
        sigma_log = np.log(nonzero).std()
    else:
        mu_log, sigma_log = 0, 1
    
    return p_zero, mu_log, sigma_log

def bayesian_score_imputation(data, n_samples=5000):
    """
    Full Bayesian imputation using score information
    """
    # Separate periods
    p1 = data[data['period'] == '2001-2005'].set_index('node')
    p2 = data[data['period'] == '2006-2010'].set_index('node')
    
    # Get matched observations
    matched_nodes = p1.index.intersection(p2.index)
    matched_p1 = p1.loc[matched_nodes, 'score'].dropna()
    matched_p2 = p2.loc[matched_nodes, 'score'].dropna()
    
    # Fit distributions for each period
    print("Fitting period 1 distribution...")
    p1_params = fit_zero_inflated_model(p1['score'])
    print(f"  P(zero) = {p1_params[0]:.2f}")
    print(f"  Non-zero: log-normal(μ={p1_params[1]:.2f}, σ={p1_params[2]:.2f})")
    
    print("\nFitting period 2 distribution...")
    p2_params = fit_zero_inflated_model(p2['score'])
    print(f"  P(zero) = {p2_params[0]:.2f}")
    print(f"  Non-zero: log-normal(μ={p2_params[1]:.2f}, σ={p2_params[2]:.2f})")
    
    # Model score transition for matched entities
    # Simple approach: score_p1 = alpha * score_p2 + noise
    # (accounts for scale difference)
    
    valid_matches = matched_p1.index.intersection(matched_p2.index)
    if len(valid_matches) > 1:
        s1 = matched_p1[valid_matches].values
        s2 = matched_p2[valid_matches].values
        
        # Filter to non-zero pairs for regression
        nonzero_mask = (s1 > 0) & (s2 > 0)
        if nonzero_mask.sum() > 1:
            # Linear regression in log space
            log_s1 = np.log(s1[nonzero_mask] + 1)
            log_s2 = np.log(s2[nonzero_mask] + 1)
            
            slope, intercept, r_val, _, _ = stats.linregress(log_s2, log_s1)
            residual_std = np.std(log_s1 - (slope * log_s2 + intercept))
            
            print(f"\nScore transition model (log scale):")
            print(f"  log(score_p1) = {slope:.3f} * log(score_p2) + {intercept:.3f}")
            print(f"  Residual std: {residual_std:.3f}")
            print(f"  R²: {r_val**2:.3f}")
        else:
            # Fallback: assume proportional scaling
            slope = 500 / 600  # Scale ratio
            intercept = 0
            residual_std = 1.0
    
    # Impute missing scores
    results = {}
    
    for node in data['node'].unique():
        if pd.isna(p1.loc[node, 'score']) and not pd.isna(p2.loc[node, 'score']):
            # Node missing in period 1, present in period 2
            score_p2 = p2.loc[node, 'score']
            
            if score_p2 == 0:
                # If zero in P2, use Bayes' theorem for P(was zero in P1)
                # Estimate transition probabilities from matched entities
                
                # Count transitions in matched data
                zero_to_zero = ((s1 == 0) & (s2 == 0)).sum()
                nonzero_to_zero = ((s1 > 0) & (s2 == 0)).sum()
                zero_to_nonzero = ((s1 == 0) & (s2 > 0)).sum()
                
                # Estimate P(P2=0 | P1=0) and P(P2=0 | P1>0)
                if (zero_to_zero + zero_to_nonzero) > 0:
                    p_stay_zero = zero_to_zero / (zero_to_zero + zero_to_nonzero)
                else:
                    p_stay_zero = 0.9  # Default: assume high persistence
                
                if (nonzero_to_zero + (s1 > 0).sum() - nonzero_to_zero) > 0:
                    p_drop_to_zero = nonzero_to_zero / (s1 > 0).sum()
                else:
                    p_drop_to_zero = 0.1  # Default: assume low dropout
                
                # Bayes' theorem
                prior_zero = p1_params[0]
                likelihood_zero = p_stay_zero
                likelihood_nonzero = p_drop_to_zero
                
                prob_was_zero = (likelihood_zero * prior_zero) / (
                    likelihood_zero * prior_zero + 
                    likelihood_nonzero * (1 - prior_zero)
                )
                
                print(f"  Zero in P2 transition probs:")
                print(f"    P(stay at 0) = {p_stay_zero:.2f}")
                print(f"    P(drop to 0) = {p_drop_to_zero:.2f}")
                print(f"    → P(was 0 in P1 | is 0 in P2) = {prob_was_zero:.2f}")
                
                samples = []
                for _ in range(n_samples):
                    if np.random.rand() < prob_was_zero:
                        samples.append(0)
                    else:
                        samples.append(np.exp(np.random.normal(
                            p1_params[1], p1_params[2]
                        )))
                samples = np.array(samples)
            else:
                # Non-zero in P2: use transition model
                log_s2 = np.log(score_p2 + 1)
                predicted_log_s1 = slope * log_s2 + intercept
                
                # Sample with uncertainty
                log_s1_samples = np.random.normal(
                    predicted_log_s1,
                    residual_std * 1.5,  # Inflate for missing data
                    n_samples
                )
                samples = np.exp(log_s1_samples) - 1
                samples = np.clip(samples, 0, 500)  # Respect period 1 range
            
            results[node] = {
                'period_2_score': score_p2,
                'imputed_samples': samples,
                'mean': samples.mean(),
                'median': np.median(samples),
                'ci_low': np.percentile(samples, 2.5),
                'ci_high': np.percentile(samples, 97.5)
            }
    
    return results

# Run imputation
results = bayesian_score_imputation(data)

# Display results
for node, result in results.items():
    print(f"\nNode {node}:")
    print(f"  Observed P2 score: {result['period_2_score']:.1f}")
    print(f"  Imputed P1 score: {result['mean']:.1f} "
          f"[{result['ci_low']:.1f}, {result['ci_high']:.1f}]")

# Convert to ranks (for each sample)
def samples_to_rank_distribution(data, results, n_samples=5000):
    """Convert score samples to rank distributions"""
    p1 = data[data['period'] == '2001-2005'].set_index('node')
    
    rank_samples = {node: [] for node in results.keys()}
    
    for sample_idx in range(n_samples):
        # Get scores for this sample
        scores = {}
        for node in data['node'].unique():
            if not pd.isna(p1.loc[node, 'score']):
                scores[node] = p1.loc[node, 'score']
            elif node in results:
                scores[node] = results[node]['imputed_samples'][sample_idx]
        
        # Rank them (higher score = better rank)
        sorted_nodes = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        ranks = {node: rank+1 for rank, (node, score) in enumerate(sorted_nodes)}
        
        for node in results.keys():
            rank_samples[node].append(ranks[node])
    
    return rank_samples

rank_distributions = samples_to_rank_distribution(data, results)

for node, ranks in rank_distributions.items():
    print(f"\nNode {node} - Rank distribution in period 1:")
    print(f"  Mean rank: {np.mean(ranks):.1f}")
    print(f"  Median rank: {np.median(ranks):.0f}")
    print(f"  95% CI: [{np.percentile(ranks, 2.5):.0f}, "
          f"{np.percentile(ranks, 97.5):.0f}]")

Fitting period 1 distribution...
  P(zero) = 0.25
  Non-zero: log-normal(μ=4.90, σ=0.94)

Fitting period 2 distribution...
  P(zero) = 0.20
  Non-zero: log-normal(μ=4.99, σ=1.03)

Score transition model (log scale):
  log(score_p1) = 0.800 * log(score_p2) + 0.990
  Residual std: 0.148
  R²: 0.975

Node E:
  Observed P2 score: 200.0
  Imputed P1 score: 191.5 [121.3, 285.6]

Node E - Rank distribution in period 1:
  Mean rank: 2.0
  Median rank: 2
  95% CI: [2, 2]


-> This would work as a novelty measure perhaps (we would then add one paper at each and then compare the impact of it)? But for the analysis for the paper, I don't think this should be included? (The way we divided the periods makes it complicated to impute ; we can impute all the missing ones in one go, but then more assumptions need to be made?)